In [5]:
import pandas as pd
import requests

In [6]:
data = pd.read_csv("NewLinks2017-12-03 20:28:18.862583.csv")

In [7]:
history = data[data.key == 'history'].drop_duplicates()

In [8]:
history.sort_values(['team'], inplace = True)

In [9]:
history_list = history.to_dict(orient='record')

In [10]:
r = requests.get(history_list[0]['link'])

In [11]:
tables = pd.read_html(r.content)

In [12]:
len(tables)

1

In [33]:
tables[0][:-1]

,Year,Head Coaches,Division,Conference,Wins,Losses,Ties,WL%,Notes
0,2017-18,Troy Calhoun,FBS,Mountain West,5,7,0,0.417,NaN
1,2016-17,Troy Calhoun,FBS,Mountain West,10,3,0,0.769,NaN
2,2015-16,Troy Calhoun,FBS,Mountain West,8,6,0,0.571,NaN
3,2014-15,Troy Calhoun,FBS,Mountain West,10,3,0,0.769,NaN
4,2013-14,Troy Calhoun,FBS,Mountain West,2,10,0,0.167,NaN
5,2012-13,Troy Calhoun,FBS,Mountain West,6,7,0,0.462,NaN
6,2011-12,Troy Calhoun,FBS,Mountain West,7,6,0,0.538,Military Bowl
7,2010-11,Troy Calhoun,FBS,Mountain West,9,4,0,0.692,Independence Bowl
8,2009-10,Troy Calhoun,FBS,Mountain West,8,5,0,0.615,Armed Forces Bowl
9,2008-09,Troy Calhoun,FBS,Mountain West,8,5,0,0.615,Armed Forces Bowl
